In [5]:
from keras.models import Model
from keras import Input, layers

input_tensor = Input(shape=(32,))
dense = layers.Dense(32, activation='relu')  # layer를 function으로 간주
output_tensor = dense(input_tensor)             # dense function에 input_tensor를 입력하여 출력을 얻어냄
model = Model(input_tensor, output_tensor)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 32)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 32)                1056      
Total params: 1,056
Trainable params: 1,056
Non-trainable params: 0
_________________________________________________________________


In [6]:
from keras.models import Sequential, Model
from keras import layers
from keras import Input

seq_model = Sequential()
seq_model.add(layers.Dense(32, activation='relu', input_shape=(64,)))
seq_model.add(layers.Dense(32, activation='relu'))
seq_model.add(layers.Dense(10, activation='softmax'))
#위에 말고 아래처럼 해도 같은 구조! 
#tensor를 인자로 하여 합성함수처럼 layer를 정의
input_tensor = Input(shape=(64,)) #
x = layers.Dense(32, activation='relu')(input_tensor) #input_tensor가 인자
x = layers.Dense(32, activation='relu')(x) #다시 x를 인자로 
output_tensor = layers.Dense(10, activation='softmax')(x) #그 x를 다시 dense함수의 인자로 
model = Model(input_tensor, output_tensor)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 64)                0         
_________________________________________________________________
dense_13 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_14 (Dense)             (None, 32)                1056      
_________________________________________________________________
dense_15 (Dense)             (None, 10)                330       
Total params: 3,466
Trainable params: 3,466
Non-trainable params: 0
_________________________________________________________________


In [7]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
import numpy as np
x_train = np.random.random((1000, 64))
y_train = np.random.random((1000, 10))
model.fit(x_train, y_train, epochs=10, batch_size=128)
score = model.evaluate(x_train, y_train)

Epoch 1/10
1000/1000 [==============================] - 0s 238us/step - loss: 11.6981
Epoch 2/10
1000/1000 [==============================] - 0s 13us/step - loss: 11.5599
Epoch 3/10
1000/1000 [==============================] - 0s 11us/step - loss: 11.5418
Epoch 4/10
1000/1000 [==============================] - 0s 14us/step - loss: 11.5321
Epoch 5/10
1000/1000 [==============================] - 0s 13us/step - loss: 11.5253
Epoch 6/10
1000/1000 [==============================] - 0s 13us/step - loss: 11.5214
Epoch 7/10
1000/1000 [==============================] - 0s 13us/step - loss: 11.5185
Epoch 8/10
1000/1000 [==============================] - 0s 13us/step - loss: 11.5165
Epoch 9/10
1000/1000 [==============================] - 0s 13us/step - loss: 11.5141
Epoch 10/10
1000/1000 [==============================] - 0s 48us/step


## Multiple Input

In [44]:
from keras.models import Model
from keras import layers
from keras import Input

text_vocabulary_size = 10000
question_vocabulary_size = 10000
answer_vocabulary_size = 500

text_input = Input(shape=(None,), dtype='int32', name='text')
embedded_text = layers.Embedding(64, text_vocabulary_size)(text_input)
encoded_text = layers.LSTM(32)(embedded_text)

question_input = Input(shape=(None,), dtype='int32', name='question')
embedded_question = layers.Embedding(32, question_vocabulary_size)(question_input)
encoded_question = layers.LSTM(16)(embedded_question)

concatenated = layers.concatenate([encoded_text, encoded_question], axis=-1) #output tensor끼리 concat
answer_output = layers.Dense(answer_vocabulary_size, activation='softmax')(concatenated)
model = Model([text_input, question_input], answer_output) #입력이 2개, 출력이 1개 나오도록 모델 학습
model.summary()
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text (InputLayer)               (None, None)         0                                            
__________________________________________________________________________________________________
question (InputLayer)           (None, None)         0                                            
__________________________________________________________________________________________________
embedding_11 (Embedding)        (None, None, 10000)  640000      text[0][0]                       
__________________________________________________________________________________________________
embedding_12 (Embedding)        (None, None, 10000)  320000      question[0][0]                   
__________________________________________________________________________________________________
lstm_11 (L

In [40]:
import numpy as np
num_samples = 1000
max_length = 100
text          = np.random.randint(1, text_vocabulary_size,          size=(num_samples, max_length))
question = np.random.randint(1, question_vocabulary_size, size=(num_samples, max_length))
answers  = np.random.randint(1, answer_vocabulary_size,    size=(num_samples, max_length))
model.fit([text, question], answers, epochs=10, batch_size=128)
#model.fit({'text': text, 'question': question}, answers, epochs=10, batch_size=128)

ValueError: Error when checking target: expected dense_19 to have shape (500,) but got array with shape (100,)

## Multiple Output

In [45]:
from keras import layers
from keras import Input
from keras.models import Model

vocabulary_size = 50000
num_income_groups = 10
posts_input = Input(shape=(None,), dtype='int32', name='posts')
embedded_posts = layers.Embedding(256, vocabulary_size)(posts_input)

x = layers.Conv1D(128, 5, activation='relu')(embedded_posts) #text니까 conv2d아니고 conv1d!
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation='relu')(x) #conv1d가 끝난 x를 dense로! 

##시험문제##
age_prediction = layers.Dense(1, name='age')(x)  #dense layer의 node가 하나임! activation없어! --> 수치예측이므로! 
income_prediction = layers.Dense(num_income_groups, activation='softmax', name='income')(x) #softmax? income이 얼마인지가 아니라 income group을 예측하는 거라서!
gender_prediction = layers.Dense(1, activation='sigmoid', name='gender')(x) #남 or여이므로 sigmoid!
model = Model(posts_input, [age_prediction, income_prediction, gender_prediction])

#model.compile(optimizer='rmsprop', loss=['mse', 'categorical_crossentropy', 'binary_crossentropy'])
#model.compile(optimizer='rmsprop', loss={'age': 'mse', 'income': 'categorical_crossentropy', 'gender': 'binary_crossentropy'})
#model.compile(optimizer='rmsprop', loss=['mse', 'categorical_crossentropy', 'binary_crossentropy'], loss_weights=[0.25, 1., 10.])
model.compile(optimizer='rmsprop', loss={'age': 'mse', 'income': 'categorical_crossentropy', 'gender': 'binary_crossentropy'}, loss_weights={'age': 0.25,
'income': 1., 'gender': 10.}) #age는 수치니까 mse incomd은 multicateogory니까 gender은 binary니까 출력되는 값들을 정규화시켜주기 위해 loss_weight필요!

#model.fit(posts, [age_targets, income_targets, gender_targets], epochs=10, batch_size=64)
model.fit(posts, {'age': age_targets, 'income': income_targets, 'gender': gender_targets}, epochs=10, batch_size=64)

NameError: name 'posts' is not defined

# Call-back

In [ ]:
from keras import models, layers
from keras.datasets import mnist
from keras.utils import np_utils

callbacks_list = [  #callback을 지정하는 문장들 
    keras.callbacks.EarlyStopping( #조기 종료를 하고자 하는데 기준은 acc값이 epoch10단계 사이의 변화가 별로 없으면! 
    monitor='acc',
    patience=10,
),
keras.callbacks.ModelCheckpoint( #데이터가 흘러가다가 file로 기록하고 싶을 때
    filepath='my_model.h5',
    monitor='val_loss', #val_loss값을 기준으로 
    save_best_only=True, #loss값이 best라고 평가될때만 기록하라!
),
keras.callbacks.ReduceLROnPlateau( #Plateau상에서 LR(학습율)을 줄여 진행속도를 느리게 하라.  
monitor='val_loss',  #val_loss값이 점점 줄다가 plateau(평지)로 오면(=감소가 없으면) 
factor=0.1, #LR * 0.1! 0.1이 0.001이 됨. 
patience=3, #3단계동안 변화가 없으면! learning rate을 줄여라. 
)]

(X_train, Y_train), (X_test, Y_test) = mnist.load_data()
X_train = X_train.reshape(60000, 784).astype('float32')/255.0
X_test = X_test.reshape(10000, 784).astype('float32')/255.0
Y_train = np_utils.to_categorical(Y_train)
Y_test = np_utils.to_categorical(Y_test)

model = models.Sequential()
model.add(layers.Dense(units=64, input_dim=28*28, activation='relu'))
model.add(layers.Dense(units=64, input_dim=28*28, activation='relu'))
model.add(layers.Dense(units=10, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
#fit할 때 call back 지정-> epoch 50까지 가지 말고 학습데이터에 대한 accuracy변화가 없으면 멈춰라. 
model.fit(X_train, Y_train, epochs=50, batch_size=32, callbacks=callbacks_list, validation_data=(X_test, Y_test))